In [1]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model, Template
from buildingmotif.namespaces import bind_prefixes, OWL, RDFS, RDF, S223
from buildingmotif.model_builder import TemplateBuilderContext
from rdflib import Namespace, URIRef, Literal, Graph, BNode
import glob
from typing import List

In [2]:
# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://", shacl_engine='topquadrant')

# create the model w/ a namespace
BLDG = Namespace("urn:nrel_example/")
bldg = Model.create(BLDG)

# enable pretty-printing of URLs for Building entities
bind_prefixes(bldg.graph)
bldg.graph.bind("bldg", BLDG)

In [3]:
# load the S223 library and the QUDT libraries
s223 = Library.load(ontology_graph=r"../libraries/ashrae/223p/ontology/223p.ttl")
qudt_libs = []
for filename in glob.glob("../libraries/qudt/*.ttl"):
    qudt_libs.append(Library.load(ontology_graph=filename))

2024-04-15 00:09:34,353 | rdflib.term |  WARNING: Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/1999/02/22-rdf-syntax-ns#HTML, Converter=<function _parse_html at 0x110db77e0>
Traceback (most recent call last):
  File "/Users/gabe/src/NREL/BuildingMOTIF/.venv/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "/Users/gabe/src/NREL/BuildingMOTIF/.venv/lib/python3.11/site-packages/rdflib/term.py", line 1663, in _parse_html
    result: xml.dom.minidom.DocumentFragment = parser.parseFragment(lexical_form)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gabe/src/NREL/BuildingMOTIF/.venv/lib/python3.11/site-packages/html5lib/html5parser.py", line 312, in parseFragment
    self._parse(stream, True, *args, **kwargs)
  File "/Users/gabe/src/NREL/BuildingMOTIF/.venv/lib/python3.11/site-packages/htm

To use the Model Builder interface, we must first create a Context and load some templates into it. The Context keeps track of the parameters and dependencies of the templates. This reduces the amount of bookkeeping required in the user application.

In [4]:
# load in the NREL template library
nrel_lib = Library.load(directory="../libraries/ashrae/223p/nrel-templates/")
# create a context for our template builder
context = TemplateBuilderContext(BLDG)
context.add_templates_from_library(nrel_lib)

Here is what basic use of the Model Builder looks like. First, create an "empty" instance of a template by indexing into the context object.

```python
mau = context["makeup-air-unit"]
```

Here, `mau` is a copy of the "makeup-air-unit" template from the NREL 223P library. We can see its parameters like this:

```python
print(mau.parameters)
```

We bind to parameters by indexing into the template copy.

```python
mau["name"] = BLDG["MAU1"] 
mau["air-supply"] = BLDG["MAU1_AIR_SUPPLY"]
```

We have bound 2 parameters (`name` and `air-supply`) to 2 values in the BLDG namespace; essentially, we have given these parameters *names* given by URIs. Agove, the name of the makeup air unit is `urn:nrel_example/MAU1`.

Here, we create a new instance of the "junction" template and give it a name

```python
junction = context["junction"]
junction["name"] = "MAU_SUPPLY_JUNCTION"
```

Note that we didn't wrap the name in a namespace (e.g. `BLDG["MAU_SUPPLY_JUNCTION"]`). Model Builder knows what the default namespace is (it is passed into the Context constructor), so strings assigned to parameters will be turned automatically into IRIs. To assign a Literal value to a parameter, make sure the right-hand side of the assignment is an `rdflib.Literal` object.

---

Next, we want to connect a Duct from the air supply of the MAU to one of the inlets of the junction. One way we could do this is by just assigning our chosen entities to the Duct template's parameters. However, we have two problems.

First, it may be difficult to remember or keep track of which names one wants to re-use. Especially as models get more complicated, having to keep track of the values bound to parameters can require significant bookkeeping.

Second, our junction's connection point doesn't have a name yet. We would like to avoid inventing a name for the connection point purely to refer to later, especially since the names of connection points rarely need to be significant.

To address these issues, the Model Builder allows parameters to be bound by reference rather than by value.

```python
# duct to connect mau air suply to junction in1
duct = context["duct"]
duct["name"] = "mau_air_supply_duct"
duct["a"] = mau["air-supply"]
duct["b"] = junction["in1"]
```

`duct["a"] = mau["air-supply"]` assigns a parameter (`duct["a"]`) to the same value that `mau["air-supply"]` is bound to. This behavior is triggered when the right-hand side of an assignment is a parameter which has already been bound.

`duct["b"] = junction["in1"]` binds the two parameters together. If one is bound to a value, the other will automatically be bound to that value too. If neither are bound to specific values, then the Context will create a unique name for them to share.

In [5]:
# create a template instance
mau = context["makeup-air-unit"]
mau["name"] = BLDG["MAU"]
mau["air-supply"] = BLDG["MAU_AIR_SUPPLY"]

junction = context["junction"]
junction["name"] = "MAU_SUPPLY_JUNCTION"

# duct to connect mau air suply to junction in1
duct = context["duct"]
duct["name"] = "mau_air_supply_duct"
duct["a"] = mau["air-supply"]
duct["b"] = junction["in1"]

Below are some helper methods for some common tasks: constructing spaces and attaching them to HVAC zones, various equipment constructors, etc. For some ontologies, like 223P, it can be helpful to have these kinds of "constructor" methods which take care of connecting the desired components together.

In [6]:
# space cache
spaces = {}

# helper functions for building the model
def ensure_space(space_name, zone):
    if space_name not in spaces:
        space = context["hvac-space"]
        space["name"] = space_name
        spaces[space_name] = space

        link = context["hvac-zone-contains-space"]
        link["name"] = zone
        link["domain-space"] = space_name
    return spaces[space_name]

def make_lab_vav(vav_name: str, space_name: str, zone: str, junction_cp: str, vav_template_name: str):
    vav = context[vav_template_name]
    vav["name"] = vav_name

    # connect vav['air-in'] to the junction with a duct
    duct = context["duct"]
    duct["name"] = f"junction_{vav_name}_duct"
    duct["a"] = junction[junction_cp]
    duct["b"] = vav["air-in"]

    ensure_space(space_name, zone)

    # connect vav['air-out'] to the space through a duct
    duct = context["duct"]
    duct["name"] = f"{vav_name}_duct"
    duct["a"] = vav["air-out"]
    duct["b"] = spaces[space_name]["in"]


def make_fcu(fcu_name: str, space_name: str, zone: str, junction_cp: str):
    fcu = context["fcu"]
    fcu["name"] = fcu_name

    # connect fcu 'in' to the junction with a duct
    duct = context["duct"]
    duct["name"] = f"junction_{fcu_name}_duct"
    duct["a"] = junction[junction_cp]
    duct["b"] = fcu["in"]

    # create the space if it doesn't exist
    ensure_space(space_name, zone)

    # connect fcu 'out' to the space through a duct
    duct = context["duct"]
    duct["name"] = f"{fcu_name}_duct"
    duct["a"] = fcu["out"]
    duct["b"] = spaces[space_name]["in"]


def make_multiple_vavs(vav_names: List[str], space_name: str, zone: str, junction_cp: str, vav_template_name: str, space_cps: List[str]):
    vavs = []
    for vav_name in vav_names:
        vav = context[vav_template_name]
        vav["name"] = vav_name
        vavs.append(vav)

    # make an upstream junction for the VAVs
    junction = context["junction"]
    junction["name"] = f"{space_name}_junction"
    # create a duct connecting junction_cp to the junction's in1
    duct = context["duct"]
    duct["name"] = f"junction_{space_name}_duct"
    duct["a"] = junction[junction_cp]
    duct["b"] = junction["in1"]


    # create the space if it doesn't exist, add it to the space cache, and connect it to the zone
    # make sure to create teh connection points from space_cps
    ensure_space(space_name, zone)

    # for each space_cp associate it with the space using hasConnectionPoint
    for space_cp in space_cps:
        cp = context["air-inlet-cp"]
        cp["name"] = space_cp
        spaces[space_name].template.body.add((spaces[space_name]["name"], S223.hasConnectionPoint, cp["name"]))

def make_multiple_fcus(fcu_names: List[str], space_name: str, zone: str, junction_cp: str, space_cps: List[str]):
    fcus = []
    for fcu_name in fcu_names:
        fcu = context['fcu']
        fcu["name"] = fcu_name
        fcus.append(fcu)

    # make an upstream junction for the VAVs
    junction = context["junction"]
    junction["name"] = f"{space_name}_junction"
    # create a duct connecting junction_cp to the junction's in1
    duct = context["duct"]
    duct["name"] = f"junction_{space_name}_duct"
    duct["a"] = junction[junction_cp]
    duct["b"] = junction["in1"]


    # create the space if it doesn't exist, add it to the space cache, and connect it to the zone
    # make sure to create teh connection points from space_cps
    ensure_space(space_name, zone)

    # for each space_cp associate it with the space using hasConnectionPoint
    for space_cp in space_cps:
        cp = context["air-inlet-cp"]
        cp["name"] = space_cp
        spaces[space_name].template.body.add((spaces[space_name]["name"], S223.hasConnectionPoint, cp["name"]))


    # for each vav, connect the junction's outX to the vav's air-in with a duct
    # where X is the index of the vav in vav_names
    for i, fcu in enumerate(fcus):
        duct = context["duct"]
        duct["name"] = f"junction_{space_name}_fcu{i+1}_duct"
        duct["a"] = junction[f"out{i + 1}"]
        duct["b"] = fcu["in"]

    # for each vav, connect the vav's air-out to the space's respective 'inlet' inside space_cps
    for i, fcu in enumerate(fcus):
        duct = context["duct"]
        duct["name"] = f"{fcu_names[i]}_space_duct"
        duct["a"] = fcu["out"]
        duct["b"] = space_cps[i]

def make_exhaust_fan(name: str, junction_cp: str, space_name: str, space_cp: str):
    ef = context["exhaust-fan"]
    ef["name"] = name
    # ef 'out' connects to the junction inlet via a duct
    duct = context["duct"]
    duct["name"] = f"junction_ef{name}_duct"
    duct["a"] = ef["out"]
    duct["b"] = eau_junction[junction_cp]
    # ef 'in' connects from the space via a duct
    duct = context["duct"]
    duct["name"] = f"ef{name}_space_duct"
    duct["a"] = spaces[space_name][space_cp]
    duct["b"] = ef["in"]


In [7]:
# call our constructors to create the equipment, zones, and rooms

make_lab_vav("VAV-103", "science-lab1", "science-lab", "out1", "lab-vav-reheat")
make_lab_vav("VAV-104", "science-lab2", "science-lab", "out2", "lab-vav-reheat")
make_lab_vav("VAV-101", "rm101", "common-space", "out5", "lab-vav-reheat")
make_lab_vav("VAV-107", "bathrms", "common-space", "out6", "lab-vav-reheat")
make_lab_vav("VAV-123", "science-lab3", "science-lab", "out3", "lab-vav-reheat")

make_multiple_vavs(["VAV-122a", "VAV-122b"], "science-lab4", "science-lab", "out4", "lab-vav-reheat", ["in1", "in2"])
make_multiple_vavs(["VAV-125a", "VAV-125b"], "corridor", "common-space", "out7", "lab-vav-reheat", ["corridor-in1", "corridor-in2"])

make_fcu("fcu109", "electricalRoom", "common-space", "out9")
make_fcu("fcu110", "ITRoom", "common-space", "out10")
make_fcu("fcu111", "office1", "common-space", "out11")
make_fcu("fcu118", "office2", "common-space", "out12")
make_fcu("fcu119", "conferenceRoom1", "common-space", "out13")
make_fcu("fcu120", "conferenceRoom2", "common-space", "out14")
make_fcu("fcuRO1", "MechanicalRoom", "common-space", "out15")

make_multiple_fcus(['fcu125a', 'fcu125b'], 'corridor', 'common-space', 'out16', ['corridor-in3', 'corridor-in4'])

In [8]:
# make eau
eau = context["exhaust-air-unit"]
eau["name"] = BLDG["EAU"]
eau["air-exhaust"] = BLDG["EAU_AIR_EXHAUST"]
eau["return-air"] = BLDG["EAU_AIR_RETURN"]

# eau junction
eau_junction = context["junction"]
eau_junction["name"] = BLDG["EAU_JUNCTION"]

# connect eau air return to the junction with a duct
duct = context["duct"]
duct["name"] = "eau_air_return_duct"
duct["a"] = eau["return-air"]
duct["b"] = eau_junction["out1"]

ensure_space("biosafety-cabinet-space", "biosafety-cabinet")

make_exhaust_fan("ef4", "in1", "biosafety-cabinet-space", "out")
make_exhaust_fan("ef5", "in2", "biosafety-cabinet-space", "out2")


When you are done building the model, call the `compile()` method on the context to evaluate all of the templates. This will drop all optional parameters and create names for any required parameters for which you did not provide values.

In [9]:
bldg.add_graph(context.compile())
bldg.graph.serialize("test.ttl", format="turtle")

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:390: UserWarning: Parameters "evaporative-cooler-evap-cool-pump-2stage-vfd-pwr, pre-filter, heating-coil-water-in-mapsto, c1, air-supply-mapsto, evaporative-cooler-in, cooling-coil-valve-in, evaporative-cooler-evap-cool-fill-valve-in-mapsto, final-filter-out, evaporative-cooler-out, cooling-coil-pump-in-mapsto, supply-fan-oa-flow-switch, evaporative-cooler-evap-cool-pump-2stage-in-mapsto, MAU-HRC-entering-air-temp, evaporative-cooler-evap-cool-pump-2stage-vfd-frq, cooling-coil-pump-out, evaporative-cooler-evap-cool-pump-2stage-in, final-filter-out-mapsto, evaporative-cooler-evap-cool-pump-2stage-vfd-cur, heating-coil-valve, heating-coil-valve-feedback, cooling-coil-entering-air-temp, cooling-coil-valve-out-mapsto, cooling-coil-valve, cooling-coil-pump-onoff-cmd, heating-coil-air-in, evaporative-cooler-evap-cool-sump-tank, sa_pressure_sensor, pre-filter-out, oad, pre-filter-in-mapsto, evaporative-cooler-evap-cool-p

<Graph identifier=d9c19699-d682-4285-9619-4c0c9636485c (<class 'rdflib.graph.Graph'>)>

In [10]:
res = bldg.validate([s223.get_shape_collection(), *[x.get_shape_collection() for x in qudt_libs]], error_on_missing_imports=False)

2024-04-15 00:12:26,557 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.w3.org/ns/shacl# from Libraries (No row was found when one was required). Trying shape collections
2024-04-15 00:12:26,570 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.w3.org/ns/shacl# from Libraries. Trying shape collections
2024-04-15 00:12:26,571 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.linkedmodel.org/schema/dtype from Libraries (No row was found when one was required). Trying shape collections
2024-04-15 00:12:26,583 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.linkedmodel.org/schema/dtype from Libraries. Trying shape collections
2024-04-15 00:12:26,584 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.linkedmodel.org/schema/vaem from Libraries (No row was found w

In [11]:
# print(res.report_string)
print(len(res.report))
print(f"Valid? {res.valid}")

56863
Valid? 0


In [12]:
for key, diffs in res.diffset.items():
    print(key)
    for d in diffs:
        print(d.reason())

http://qudt.org/vocab/unit/MegaHZ
http://qudt.org/vocab/unit/MegaHZ needs between None and 1 uses of path http://qudt.org/schema/qudt/conversionMultiplier
http://qudt.org/vocab/unit/MegaHZ needs between None and 1 uses of path http://qudt.org/schema/qudt/conversionMultiplier
http://qudt.org/vocab/unit/MegaHZ needs between None and 1 uses of path http://purl.org/dc/terms/description
http://qudt.org/vocab/unit/HectoBAR
http://qudt.org/vocab/unit/HectoBAR needs between None and 1 uses of path http://qudt.org/schema/qudt/conversionMultiplier
http://qudt.org/vocab/unit/HectoBAR needs between None and 1 uses of path http://qudt.org/schema/qudt/conversionMultiplier
http://qudt.org/vocab/unit/N-PER-MilliM2
http://qudt.org/vocab/unit/N-PER-MilliM2 needs between None and 1 uses of path http://qudt.org/schema/qudt/conversionMultiplier
http://qudt.org/vocab/unit/N-PER-MilliM2 needs between None and 1 uses of path http://qudt.org/schema/qudt/conversionMultiplier
http://qudt.org/vocab/quantitykind/R